In [1]:
#Сложение точек и умножение на скаляр

#first_point and second_point  - points elliptic curve, a-coef A Elliptic Curve,  return point elliptic curve

def add(first_point, second_point, a): 
    
        if(first_point == E[0]):
            return second_point
        
        if(second_point == E[0]):
            return first_point;
        
        if first_point[0] != second_point[0]: #if x1!=x2
            obliquity = (second_point[1] - first_point[1])/(second_point[0] - first_point[0])
            x = pow(obliquity, 2) - first_point[0] - second_point[0]
            y = obliquity*(first_point[0] - x) - first_point[1]
            return E(x,y)
        elif (first_point[1] == (-second_point[1])):
            print("0-point")
            return E[0]
        else:
            obliquity = (3*pow(first_point[0],2) + a)/(2*first_point[1])
            x = pow(obliquity, 2) - 2*first_point[0]
            y = obliquity*(first_point[0]-x) - first_point[1]
            return E(x,y)

        
#return point        
def mul(point, scalar,a):
    result = point
    for i in range(scalar-1):
        result = add(result,point,a)
    return result


In [86]:
R = GF(11)
E = EllipticCurve(R, [1,3])
p = E(0,5)
# print(p)
q = E(0,5)
# print(q)
print(p+q)
print(add(p,q,1))
# print(mul(p, 11,1))
# print(11*p)

(1 : 7 : 1)
(1 : 7 : 1)


In [2]:
#вычисление группы классов CL(Od) и числа классов hd
#input: D - (int)discriminant
#output: h - (int)number class hd, CL -(list) group CL 
import random
def algo1(D:int)->tuple: #D=1(mod 4)
    h = 0
    CL = []
    b = D % 2
    mod_D = D
    if(D<0):
        mod_D = -D
    B = int(sqrt(int(mod_D//3)))
    
    while(b<B):
        q = (pow(b,2)+mod_D)//4
        a = b
        if(a<=1):
            a=1
            if(pow(a,2)> q):
                b = b + 2
                continue
        while True:
            if(q % a == 0):
                c = (pow(b,2)-D)//(4*a)
                if(a==b or pow(a,2)==b or b==0):
                    h = h + 1
                    CL.append([a,b,c])
                else:
                    h = h + 2
                    CL.append([a,b,c])
                    CL.append([a,-b,c])
            a=a+1
            if(pow(a,2)>q):
                break
                
        b = b + 2
    
    return h, CL  #return hd & CL(Od)

In [3]:
#Search characteristic field 
#input: D - discriminant
#output: p-characteristic, a and b-coef
import random
def searchP(D:int)->tuple:
    low = pow(2, 254)
    high = pow(2, 256)
    print("start")
    if(D<0):
        D=-D
    first_coef = ((D+1)//4)
    a = random.randint(pow(2, 125), pow(2, 130))
    while True:
        b = random.randint(pow(2, 125), pow(2, 127))
        p = first_coef * pow(a,2) + D*a*b + D*pow(b,2)
        if(is_prime(p)):
            m1=p + 1 + a
            if(not m1&1):
                r1 = ecm.factor(m1)
                print(r1[-1])
                if r1[-1]> low and r1[-1]<high:
                    return p,a,b
            m2=p + 1 - a
            if(not m1&1):
                if(m2>0):
                    r2 = ecm.factor(m2)
                    print(r2[-1])
                    if r2[-1]>low and r2[-1]<high:
                        return p,a,b


Descrim = -20330567

p=10683863876144140607596037180633903641062677122953791254380615843457373831710272591247
a=1122163254750825614190870165471914037080
b=163837269953416404312883646079501012881
m = p + 1 - a
r = ecm.factor(m)
print(r)
#28948022309329048855892746252171976963317496166410141009864396001978282409984
#93285084633317719910916299446997712103182136941521590966740127679487571500893
#115792089237316195423570985008687907853269984665640564039457584007913129639936

[2, 2, 2, 3, 3, 1590683, 93285084633317719910916299446997712103182136941521590966740127679487571500893]


In [4]:
#input: p- characteristic field, r-simple divisor p
#output: bool - suitable or not
def PFits(p:int, r:int)->bool:
    B = 32
    k = 1
    while(k<B):
#         print(f"{k=}")
#         print(f"{pow(p,k,r)=}")
        if(pow(p,k,r) == 1):
            return False
        k+=1
    return True

In [5]:
def bezout(a, b):
    '''An implementation of extended Euclidean algorithm.
    Returns integer x, y and gcd(a, b) for Bezout equation:
        ax + by = gcd(a, b).
    '''
    x, xx, y, yy = 1, 0, 0, 1
    while b:
        q = a // b
        a, b = b, a % b
        x, xx = xx, x - xx*q
        y, yy = yy, y - yy*q
    return x
#Вычисление A и B коэф. эллиптической кривой
#input: j-invariant, p-characteristic field
#output: A and B = coef
def SolveAB(j:int, p:int)->tuple:
    inv = bezout(1728-j, p)
    
    A = (3*j*inv)%p
    B = (2*j*inv)%p
    
    return A,B
    

In [6]:
#проверка скрученная кривая или нет
#input: A,B-коэф. кривой, m-заданное количество точек, p-характеристика
#output: true or false
def twisted(A:int, B:int, m:int, p:int)->bool:
    R = GF(p)
    E = EllipticCurve(R, [A,B])
#     print(f"E={E.cardinality()}")
#     print(f"{m=}")
    if E.cardinality() == m:
        return True
    return False

#input: A,B-коэф. кривой, m-заданное количество точек, p-характеристика
#output: A,B
def check_twisted(A:int, B:int, m:int, p:int)->tuple:
    if(twisted(A,B,m,p)):
        return A,B
    else:
        #найти невычет квадратичный t
        for t in range(p):
            if kronecker(t,p)==-1 and t>100:
                break
#         print(f"{t=}")
        A = (A * pow(t,2))%p
        B = (B * pow(t,3))%p
#         print(f"{A=},{B=}")
        if(twisted(A,B,m,p)):
            return A,B
        else:
            print("error")
            return 0,0

In [17]:
import random
#Поиск x0, что x0+Ax0+B - квадратичный вычет по модулю p
#input: А и B - коэф.кривой p-характеристика, r- порядок подгруппы
#output: координаты точки x0,y0, образующей циклическую подгруппу порядка r 
def searchX(A:int,B:int,p:int, r,m):
    R = GF(p)
    E = EllipticCurve(R, [A,B])
    p = E.random_element()
    if(p.order() == r[-1]):
        return p
    else:
        print(f"{p.order()=}")
        need_mul = m // r[-1]
        print(f"{need_mul=} ")
        q = need_mul*p
        print(f"{q=}")
        print(f"{q.order()}")
        return q
        
        

In [18]:

Descrim = -20330567
# h, CL = algo1(Descrim)
# print(h)
# print(CL)
#p, a, b = searchP(Descrim)
#print(f"{p=}, {a=}, {b=}")
p=10683863876144140607596037180633903641062677122953791254380615843457373831710272591247
a=1122163254750825614190870165471914037080
b=163837269953416404312883646079501012881
m = p + 1 - a
r = ecm.factor(m)
print(r)
print(r[-1])
j = 2822285505244240609064881573451638766679524945491308869709840817974058286278389768242
A,B = SolveAB(j,p)
A,B = check_twisted(A,B,m,p)
p=searchX(A,B,p,r,m)
print("end")
print(p)


#28948022309329048855892746252171976963317496166410141009864396001978282409984
#93285084633317719910916299446997712103182136941521590966740127679487571500893
#115792089237316195423570985008687907853269984665640564039457584007913129639936

[2, 2, 2, 3, 3, 1590683, 93285084633317719910916299446997712103182136941521590966740127679487571500893]
93285084633317719910916299446997712103182136941521590966740127679487571500893
p.order()=1780643979357356767932672863438983940177112853638604666604965038211083944373059759028
need_mul=114529176 
q=(8442101807768499364151850836741976522055021671871212500217168563290060651198755836095 : 2999434091472766154562167953873736379206231138290929352545530240571187525952662573512 : 1)
93285084633317719910916299446997712103182136941521590966740127679487571500893
end
(8442101807768499364151850836741976522055021671871212500217168563290060651198755836095 : 2999434091472766154562167953873736379206231138290929352545530240571187525952662573512 : 1)


NameError: name 'y0' is not defined